#### Align reads using bowtie. This command will output a sam file (alignment file) and a .log file which contains the details on how the alignment went. (make sure you have latest version of bowtie)

In [4]:
!bowtie -S -m 1 -v 0 indexes/NC_000913.2 netseq_data/clean_Trimmed_wt-mmc.fastq.gz 2> wt_mmc_NET.log|samtools view -b -h -F 4|bedtools bamtobed -i '-' > wt_mmc_NET_aligned_2.bed 

#### Convert sam to bam and only keep reads that aligned, convert bam to bed:

In [8]:
!grep -w '+' wt_mmc_NET_aligned_2.bed|awk '{print $1"\t"$2"\t"($2+1)"\t"".""\t"$5"\t"$6}'> plus_3end_2.bed
!grep -w '-' wt_mmc_NET_aligned_2.bed|awk '{print $1"\t"($3-1)"\t"($3)"\t"".""\t"$5"\t"$6}'> minus_3end_2.bed

#### Edit bedfiles with pandas to only include the coordinate of the 3' base (needs to be double checked)

In [10]:
import pandas as pd

In [ ]:
reads=pd.read_table('wt_mmc_NET_aligned.bed',header=None)
plus_reads=reads[reads[5]=='+']
minus_reads=reads[reads[5]=='-']
plus_reads[2]=plus_reads.loc[:,1].apply(lambda x:x+1)
plus_reads.to_csv('plus_3end.bed',header=False,sep='\t',index=False)
minus_reads[1]=minus_reads.loc[:,2].apply(lambda x:x-1)
minus_reads.to_csv('minus_3end.bed',header=False,sep='\t',index=False)

In [12]:
m_3end=pd.read_table('minus_3end_2.bed',header=None)

In [17]:
m_counts=pd.DataFrame(m_3end[2].value_counts()).sort_index()

In [20]:
m_counts

,2
26,2
28,2
35,1
37,2
39,1
41,1
67,2
83,1
104,1
111,1


## Get Coverage across the whole genome for each strand

In [19]:
m_counts.reindex(range(m_counts.index.max() + 1),fill_value=0)

,2
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


#### Use updated bedifles that were just made to determine the amount of 3'end for every position in each gene

In [ ]:
bedtools coverage -d -a NC_000913.2.gtf -b plus_3end.bed > wt_mmc_NET_plus_genecov.bed
bedtools coverage -d -a NC_000913.2.gtf -b minus_3end.bed > wt_mmc_NET_minus_genecov.bed